# Walker2D

You aim in this task is to train the agent to win in Walker2D game with Actor-Critic, Advantage Actor Critic (A2C), Trust-region Policy Optimization (TRPO) or Proximal Policy Optimization (PPO).
To solve the task feel free to transform the state and reward from the environment.

**Scoring**: Calculating the average reward for 50 episodes. You goal is to gain more than 1000 points.

**Submission format**: send you notebook and trained model in **zipped** folder.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python --version

Python 3.8.16


In [3]:
! pip install PyBullet >> None

In [4]:
!pip list

Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.3.1
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.9
bokeh                         2.3.3
branca

In [5]:
import pybullet_envs
from gym import make
import numpy as np
import torch
from torch import nn
from torch.distributions import Normal, MultivariateNormal
from torch.nn import functional as F
from torch.optim import Adam
import random
import os

/usr/local/lib/python3.8/dist-packages/gym/envs/registration.py:440: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(


In [6]:
device = torch.device('cpu')

if(torch.cuda.is_available()):
    device = torch.device('cuda:0')
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")

Device set to : cpu


# Train

In [7]:
ENV_NAME = "Walker2DBulletEnv-v0"

LAMBDA = 0.95
GAMMA = 0.99

ACTOR_LR = 2e-5
CRITIC_LR = 1e-5

CLIP = 0.2
ENTROPY_COEF = 1e-2
BATCHES_PER_UPDATE = 2048
BATCH_SIZE = 64

MIN_TRANSITIONS_PER_UPDATE = 2048
MIN_EPISODES_PER_UPDATE = 4

ITERATIONS = 1000

def compute_lambda_returns_and_gae(trajectory):
    lambda_returns = []
    gae = []
    last_lr = 0.
    last_v = 0.
    for _, _, r, _, v in reversed(trajectory):
        ret = r + GAMMA * (last_v * (1 - LAMBDA) + last_lr * LAMBDA)
        last_lr = ret
        last_v = v
        lambda_returns.append(last_lr)
        gae.append(last_lr - v)

    # Each transition contains state, action, old action probability, value estimation and advantage estimation
    return [(s, a, p, v, adv) for (s, a, _, p, _), v, adv in zip(trajectory, reversed(lambda_returns), reversed(gae))]



class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        # Advice: use same log_sigma for all states to improve stability
        # You can do this by defining log_sigma as nn.Parameter(torch.zeros(...))
        ''' YOUR CODE HERE '''
        self.model =  nn.Sequential(
              nn.Linear(state_dim, 256),
              nn.ELU(),
              nn.Linear(256, 256),
              nn.ELU(),
              nn.Linear(256, action_dim))
        self.sigma = nn.Parameter(torch.ones(action_dim))

    def compute_proba(self, state, action):
        # Returns probability of action according to current policy and distribution of actions
        ''' YOUR CODE HERE '''
        mu = self.model(state)
        dist = Normal(mu, torch.exp(self.sigma))
        prob = torch.exp(dist.log_prob(action).sum(-1))
        return prob, dist

    def act(self, state):
        # Returns an action (with tanh), not-transformed action (without tanh) and distribution of non-transformed actions
        # Remember: agent is not deterministic, sample actions from distribution (e.g. Gaussian)
        ''' YOUR CODE HERE '''
        mu = self.model(state)
        dist = Normal(mu, torch.exp(self.sigma))
        action = dist.sample()

        return torch.tanh(action), action, dist


class Critic(nn.Module):
    def __init__(self, state_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ELU(),
            nn.Linear(256, 256),
            nn.ELU(),
            nn.Linear(256, 1)
        )

    def get_value(self, state):
        return self.model(state)


class PPO:
    def __init__(self, state_dim, action_dim):
        self.actor = Actor(state_dim, action_dim)
        self.critic = Critic(state_dim)
        self.actor_optim = Adam(self.actor.parameters(), ACTOR_LR)
        self.critic_optim = Adam(self.critic.parameters(), CRITIC_LR)

    def update(self, trajectories):
        transitions = [t for traj in trajectories for t in traj] # Turn a list of trajectories into list of transitions
        state, action, old_prob, target_value, advantage = zip(*transitions)
        state = np.array(state)
        action = np.array(action)
        old_prob = np.array(old_prob)
        target_value = np.array(target_value)
        advantage = np.array(advantage)
        advnatage = (advantage - advantage.mean()) / (advantage.std() + 1e-8)


        for _ in range(BATCHES_PER_UPDATE):
            idx = np.random.randint(0, len(transitions), BATCH_SIZE) # Choose random batch

            s = torch.tensor(state[idx]).float()
            a = torch.tensor(action[idx]).float()
            op = torch.tensor(old_prob[idx]).float() # Probability of the action in state s.t. old policy
            v = torch.tensor(target_value[idx]).float() # Estimated by lambda-returns
            adv = torch.tensor(advantage[idx]).float() # Estimated by generalized advantage estimation

            ''' YOUR CODE HERE '''
            # TODO: Update actor here
            # calculate ratios
            new_prob, dist = self.actor.compute_proba(s, a)
            ratio = new_prob / op

            # actor_loss
            surr_loss = ratio * adv
            clipped_surr_loss = (
                torch.clamp(ratio, 1.0 - CLIP, 1.0 + CLIP) * adv
            )

            # entropy
            entropy = dist.entropy().mean()

            actor_loss = (
                - torch.min(surr_loss, clipped_surr_loss).mean()
                - entropy * ENTROPY_COEF
            )

            self.actor_optim.zero_grad()
            actor_loss.backward(retain_graph=True)
            self.actor_optim.step()


            # TODO: Update critic here
            # critic_loss
            value = self.critic.get_value(s).flatten()
            # critic_loss = (v - value).pow(2).mean()
            critic_loss = F.smooth_l1_loss(value, v)

            # train critic
            self.critic_optim.zero_grad()
            critic_loss.backward(retain_graph=True)
            self.critic_optim.step()



    def get_value(self, state):
        with torch.no_grad():
            state = torch.tensor(np.array([state])).float()
            value = self.critic.get_value(state)
        return value.cpu().item()

    def act(self, state):
        with torch.no_grad():
            state = torch.tensor(np.array([state])).float()
            action, pure_action, distr = self.actor.act(state)
            prob = torch.exp(distr.log_prob(pure_action).sum(-1))
        return action.cpu().numpy()[0], pure_action.cpu().numpy()[0], prob.cpu().item()

    def save(self):
        torch.save(self.actor, "./drive/MyDrive/Colab Notebooks/agent.pkl")



def evaluate_policy(env, agent, episodes=5):
    returns = []
    for _ in range(episodes):
        done = False
        state = env.reset()
        total_reward = 0.

        while not done:
            state, reward, done, _ = env.step(agent.act(state)[0])
            total_reward += reward
        returns.append(total_reward)
    return returns


def sample_episode(env, agent):
    s = env.reset()
    d = False
    trajectory = []
    while not d:
        a, pa, p = agent.act(s)
        v = agent.get_value(s)
        ns, r, d, _ = env.step(a)
        trajectory.append((s, pa, r, p, v))
        s = ns
    return compute_lambda_returns_and_gae(trajectory)

In [8]:
env = make(ENV_NAME)
ppo = PPO(state_dim=env.observation_space.shape[0], action_dim=env.action_space.shape[0])
state = env.reset()
episodes_sampled = 0
steps_sampled = 0

for i in range(ITERATIONS):
    trajectories = []
    steps_ctn = 0

    while len(trajectories) < MIN_EPISODES_PER_UPDATE or steps_ctn < MIN_TRANSITIONS_PER_UPDATE:
        traj = sample_episode(env, ppo)
        steps_ctn += len(traj)
        trajectories.append(traj)
    episodes_sampled += len(trajectories)
    steps_sampled += steps_ctn

    ppo.update(trajectories)

    if (i + 1) % (ITERATIONS//100) == 0:
        rewards = evaluate_policy(env, ppo, 5)
        print(f"Step: {i+1}, Reward mean: {np.mean(rewards)}, Reward std: {np.std(rewards)}, Episodes: {episodes_sampled}, Steps: {steps_sampled}")
        ppo.save()

/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that

Step: 10, Reward mean: 19.090226625995417, Reward std: 8.417874725248836, Episodes: 1227, Steps: 20599
Step: 20, Reward mean: 43.79200333704823, Reward std: 21.835163204675734, Episodes: 1522, Steps: 41408
Step: 30, Reward mean: 55.80040976632737, Reward std: 9.578207709096326, Episodes: 1794, Steps: 62282
Step: 40, Reward mean: 57.35467100295587, Reward std: 18.436448060534502, Episodes: 2056, Steps: 83256
Step: 50, Reward mean: 63.82988005354157, Reward std: 3.2174581345465616, Episodes: 2303, Steps: 104105
Step: 60, Reward mean: 66.60376613301747, Reward std: 3.5184390522395903, Episodes: 2553, Steps: 125027
Step: 70, Reward mean: 66.89782017923889, Reward std: 14.048683497841706, Episodes: 2785, Steps: 145860
Step: 80, Reward mean: 91.51735674849218, Reward std: 16.577037232981876, Episodes: 2998, Steps: 166763
Step: 90, Reward mean: 66.24226068459888, Reward std: 22.962179238093164, Episodes: 3208, Steps: 187844
Step: 100, Reward mean: 78.53412515200732, Reward std: 8.925402821401

# Agent

In [ ]:
class Agent:
    def __init__(self):
        self.model = torch.load(__file__[:-8] + "/agent.pkl")

    def act(self, state):
        with torch.no_grad():
            state = torch.tensor(np.array(state)).float()
            ''' YOUR CODE HERE '''
            actions = self.model(state)
            return np.argmax(actions.cpu().numpy())

    def reset(self):
        pass